* There is possibly something covering the camera. Beginning in June. The day videos are still (probably) usable; the night videos are not.
* The camera has moved. Beginning in June. May need to be recalibrated.
